In [ ]:
import os
import glob
import xarray as xr

In [ ]:
DATA_DIR = "data/chlorophyll_nc_files/"

files = sorted(glob.glob(os.path.join(DATA_DIR, "*.nc")))
files


In [ ]:
datasets = []
for f in files:
    ds = xr.open_dataset(f)
    datasets.append(ds)

data = xr.concat(datasets, dim="time")
data


In [ ]:
chl = data["chlor_a"].where(data["chlor_a"] >= 0.001)
lat = data["lat"].values
lon = data["lon"].values


In [ ]:
import matplotlib.pyplot as plt

i = 0  # first month

plt.figure(figsize=(12,6))
plt.pcolormesh(lon, lat, chl.isel(time=i), cmap='turbo', shading='auto')
plt.colorbar(label="Chlorophyll (mg/m³)")
plt.title("Chlorophyll First Month")
plt.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

In [ ]:
# Downsample
step = 10

lat_ds = lat[::step]
lon_ds = lon[::step]

lon2d_ds, lat2d_ds = np.meshgrid(lon_ds, lat_ds)

lat_rad = np.radians(lat2d_ds)
lon_rad = np.radians(lon2d_ds)
R = 1

x = R * np.cos(lat_rad) * np.cos(lon_rad)
y = R * np.cos(lat_rad) * np.sin(lon_rad)
z = R * np.sin(lat_rad)


In [ ]:
chl_log_ds = []

for i in range(chl.shape[0]):
    frame = chl.isel(time=i).values[::step, ::step]

    # mask land
    frame = np.where(np.isfinite(frame), frame, 0.01)

    # log10 transform
    frame_log = np.log10(frame)

    # assign land = -3
    frame_log = np.where(frame < 0.011, -3, frame_log)

    chl_log_ds.append(frame_log)


In [ ]:
colorscale = [
    [0.00, "rgb(235,235,235)"],   # land
    [0.05, "rgb(235,235,235)"],   # land

    [0.10, "rgb(220,235,255)"],   # very light blue
    [0.30, "rgb(150,200,255)"],   # blue
    [0.50, "rgb(90,200,170)"],    # greenish blue
    [0.70, "rgb(255,230,150)"],   # yellow
    [0.90, "rgb(255,180,100)"],   # orange
    [1.00, "rgb(255,130,60)"]     # amber/orange
]



In [ ]:
import plotly.graph_objects as go

frames = []
for i, grid in enumerate(chl_log_ds):
    frames.append(
        go.Frame(
            data=[
                go.Surface(
                    x=x, y=y, z=z,
                    surfacecolor=grid,
                    cmin=-3,
                    cmax=1.3,
                    colorscale=colorscale,
                    showscale=False,
                )
            ],
            name=f"{i}",
            layout=dict(
                scene=dict(
                    camera=dict(
                        eye=dict(x=np.cos(i * 0.1), 
                                 y=np.sin(i * 0.1),
                                 z=0.5)
                    )
                )
            )
        )
    )



In [ ]:
import pandas as pd

dates = pd.date_range("2024-01-01", "2025-11-01", freq="MS").strftime("%b %Y").tolist()
len(dates)


In [ ]:
import plotly.graph_objects as go

# Build slider frames (static — no layout/camera changes)
frames_slider = [
    go.Frame(
        data=[
            go.Surface(
                x=x, y=y, z=z,
                surfacecolor=chl_log_ds[i],
                cmin=-3,
                cmax=1.3,
                colorscale=colorscale,
                showscale=False
            )
        ],
        name=f"{i}"
    )
    for i in range(len(chl_log_ds))
]

# Initial figure (first month)
fig_slider = go.Figure(
    data=[
        go.Surface(
            x=x, y=y, z=z,
            surfacecolor=chl_log_ds[0],
            cmin=-3,
            cmax=1.3,
            colorscale=colorscale,
            showscale=True,
            colorbar=dict(
                title=dict(
                    text="log10(chl mg/m³)",
                    font=dict(color="black")
                ),
                tickfont=dict(color="black")
            )
        )
    ],
    frames=frames_slider
)

# Layout with only a slider, no buttons, no animation
fig_slider.update_layout(
    title=dict(
        text="Chlorophyll Globe (2024–2025)",
        font=dict(color="black", size=20)
    ),
    width=900,
    height=900,
    paper_bgcolor="white",
    plot_bgcolor="white",
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        bgcolor="white",
        aspectmode="data",
    ),
    
    sliders=[
        dict(
            steps=[
                dict(
                    method="animate",
                    args=[[str(i)], {"mode": "immediate"}],
                    label=dates[i]
                )
                for i in range(len(chl_log_ds))
            ],
            currentvalue=dict(
                prefix="Month: ",
                font=dict(color="black")
            ),
            pad=dict(t=50)
        )
    ]
)

fig_slider.show()


In [ ]:
frames_clean = []

# smooth rotation (one full circle across all months)
angles = np.linspace(0, 2*np.pi, len(chl_log_ds))

radius = 2.3     # stable zoom level
z_tilt = 0.35     # slight tilt for aesthetics

for i, (grid, ang) in enumerate(zip(chl_log_ds, angles)):
    
    camera_eye = dict(
        x=radius * np.cos(ang),
        y=radius * np.sin(ang),
        z=z_tilt
    )
    
    frames_clean.append(
        go.Frame(
            name=str(i),
            data=[
                go.Surface(
                    x=x, y=y, z=z,
                    surfacecolor=grid,
                    cmin=-3,
                    cmax=1.3,
                    colorscale=colorscale,
                    showscale=False      # no colorbar here
                )
            ],
            layout=dict(
                scene_camera=dict(eye=camera_eye),
                annotations=[
                    dict(
                        text=dates[i],
                        x=0.5, y=0.05,
                        xref="paper", yref="paper",
                        showarrow=False,
                        font=dict(color="black", size=22)
                    )
                ]
            )
        )
    )


In [ ]:
static_colorbar = go.Surface(
    x=x, y=y, z=z,
    surfacecolor=chl_log_ds[0],
    cmin=-3,
    cmax=1.3,
    colorscale=colorscale,
    showscale=True,                  
    opacity=0.01,                    # <-- FIX (must not be 0)
    colorbar=dict(
        title=dict(text="log10(chl mg/m³)", font=dict(color="black")),
        tickfont=dict(color="black")
    )
)


In [ ]:
initial_surface = go.Surface(
    x=x, y=y, z=z,
    surfacecolor=chl_log_ds[0],
    cmin=-3,
    cmax=1.3,
    colorscale=colorscale,
    showscale=False
)


In [ ]:
fig_clean = go.Figure(
    data=[static_colorbar, initial_surface],
    frames=frames_clean
)

fig_clean.update_layout(
    title=dict(
        text="Chlorophyll Globe — 2024–2025",
        font=dict(color="black", size=22)
    ),
    width=950,
    height=950,
    paper_bgcolor="white",
    plot_bgcolor="white",

    scene=dict(
        bgcolor="white",
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode="data",
        camera=dict(eye=dict(x=2.3, y=0, z=0.35))
    ),

    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            x=0, y=-0.1,
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[
                        None,
                        dict(
                            frame=dict(duration=120, redraw=True),
                            transition=dict(duration=0),
                            fromcurrent=True
                        )
                    ]
                )
            ]
        )
    ],

    annotations=[
        dict(
            text=dates[0],
            x=0.5, y=0.05,
            xref="paper", yref="paper",
            showarrow=False,
            font=dict(color="black", size=22)
        )
    ]
)
fig_clean.update_layout(coloraxis_showscale=False)

fig_clean.show()


In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
import os

os.makedirs("frames_chl", exist_ok=True)

for i, fr in enumerate(frames_clean):

    # rotating surface trace for this frame
    rotating_surface = fr.data[0]

    if fr.layout.scene and fr.layout.scene.camera:
        camera_eye = fr.layout.scene.camera
    else:
        camera_eye = dict(eye=dict(x=2.3, y=0, z=0.35))

    annotations = fr.layout.annotations if fr.layout.annotations else []

    fig_frame = go.Figure(
        data=[
            static_colorbar,      # permanent colorbar
            rotating_surface      # visible rotating Earth
        ],
        layout=dict(
            width=950,
            height=950,
            paper_bgcolor="white",
            plot_bgcolor="white",
            annotations=annotations,
            scene=dict(
                bgcolor="white",
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False),
                aspectmode="data",
                camera=camera_eye
            )
        )
    )

    # export PNG
    pio.write_image(fig_frame, f"frames_chl/frame_{i:04d}.png", scale=2)

    print(f"Saved frame {i+1}/{len(frames_clean)}")


In [ ]:
!ffmpeg -framerate 2 -i frames_chl/frame_%04d.png \
  -pix_fmt yuv420p -vcodec libx264 -crf 18 chlorophyll.mp4
